In [ ]:
import os
import gc
import cv2
import tqdm
import glob
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
rcParams['figure.figsize'] = 10, 10

In [ ]:
for path in glob.glob('/kaggle/input/hubmap-kidney-segmentation/train/*.tiff'):
    box_size = 1024
    img = tiff.imread(path)

    if len(img.shape) == 5: img = np.transpose(img.squeeze(), (1,2,0))

    x_pad = box_size - (img.shape[0] % box_size)
    xsp = x_pad // 2
    xep = x_pad - xsp
    y_pad = box_size - (img.shape[1] % box_size)
    ysp = y_pad // 2
    yep = y_pad - ysp

    img = np.pad(img, ((xsp,xep),(ysp, yep), (0,0)), 'constant')
    img[img == 0] = 220
    mini_img = cv2.resize(img, (1024,1024))
    shifted = cv2.pyrMeanShiftFiltering(mini_img, 25, 500, 1)
    shifted = shifted.mean(axis=-1) < mini_img.mean() * 1.1
    
    group_size = 8
    idxs = np.round(np.linspace(0, mini_img.shape[0], int(mini_img.shape[0]/group_size))).astype(int)
    idys = np.round(np.linspace(0, mini_img.shape[1], int(mini_img.shape[1]/group_size))).astype(int)
    avg_img = np.zeros((len(idxs), len(idys)))

    for idx in range(len(idxs) - 1):
        for idy in range(len(idys) - 1):
            xs = idxs[idx]
            xe = idxs[idx+1]
            ys = idys[idy]
            ye = idys[idy+1]
            img_sel = shifted[xs:xe, ys:ye]
            avg_img[idx+1, idy+1] = img_sel.mean() > 0.0
# commented out due to size and image 
#     avg_img = cv2.resize(avg_img, (img.shape[1], img.shape[0]), cv2.INTER_NEAREST) > 0.0
#     plt.imshow(img)
#     plt.imshow(avg_img, cmap='jet', alpha=0.5)
    avg_img = cv2.resize(avg_img, (mini_img.shape[1], mini_img.shape[0]), cv2.INTER_NEAREST) > 0.0
    plt.imshow(mini_img)
    plt.imshow(avg_img, cmap='jet', alpha=0.5)
    plt.show()
    gc.collect()